In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# Required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from matplotlib.ticker import NullFormatter
import seaborn as sns
import math
import os
from matplotlib import rcParams

In [ ]:
# Loading the dataset

df_info = pd.read_csv("/content/drive/My Drive/Colab Notebooks/courseraforums-master/data/course_information.csv",encoding = 'unicode_escape') #details abouts the courses
df_post = pd.read_csv("/content/drive/My Drive/Colab Notebooks/courseraforums-master/data/course_posts.csv",encoding = 'unicode_escape') # details about the posts
df_thread = pd.read_csv("/content/drive/My Drive/Colab Notebooks/courseraforums-master/data/course_threads.csv",encoding = 'unicode_escape') # details about the thread
df_thread_modified = pd.read_csv("/content/drive/My Drive/Colab Notebooks/courseraforums-master/data/course_threads_modified.csv") # modified thred.csv, this dataset will be used for figure 5 and 6

Figure 1 demostrates no of threads per course.
Here thread are composed of posts and messages. We can see that the highest no of thread os for the "introduction of physics" followed by "video game learning" course.


In [ ]:
# Figure 1
rcParams['figure.figsize'] = 15.0,8.0
Fig1_df = df_info.sort_values('num_threads',ascending=False)
#df_info.plot.bar(x="course_id", y="num_threads", title="No of Threads Vs course identifier",figsize=(12,4));
sns.barplot(x= 'course_id',y = 'num_threads', data = Fig1_df, color = 'b')
plt.xticks(rotation=90, horizontalalignment="center")
plt.title("Number of threads vs. course identifier")
plt.ylabel("Number of Threads", size=14)
plt.xlabel("Course ID", size=14)



Here in the research paper, it is not clear that which column represents the no of messages. Though, in the course_post.csv file "post_id" is mentioned for each thread. So, i am considering that count of post_id columns should give the "Total no of messages"

Figure 2 A, demonstartes the relationship between size of thread and thread count.

As per the research paper, there is a decreasing trend between no of threads and and no of messages, meaning if no of threds increases no of messages will decreases (which seems not very logical, ideally there will be more no messages if more no of threads. 

As per my result, there is a positive linear trend between no of messages and no of threads, meaning no of messages will increase with incresing no of thread count.

In [ ]:
info_post_df = pd.merge(df_info, df_post, on ="course_id", how = "left")
#info_post_df.to_csv(r'C:/Users/aryac/Downloads/courseraforums-master/data/info_post.csv', index = False)

In [ ]:
Fig2a = info_post_df.groupby('num_threads').size().reset_index()
Fig2a.rename(columns={0 :'no_of_messages'}, inplace=True )
Fig2a.head()

In [ ]:
Figure2a = sns.scatterplot(x = "no_of_messages", y = "num_threads", data=Fig2a)
rcParams['figure.figsize'] = 8.0,6.0
Figure2a.set_yscale('log')
Figure2a.set_xscale('log')
plt.title("Log plot of - size of threads vs. thread count")
plt.ylabel("Number of Threads", size=14)
plt.xlabel("Thread Size (No of messages)", size=14)



Figure 2 A, demonstartes the relationship between size of thread and No of users

As per the research paper, there is a decreasing trend between no of users involved and and no of messages, meaning if no of users involvment increases no of messages will decreases (which seems not very logical, ideally there will be more no messages if more no of users are involved.

As per my result, there is a positive linear trend between no of messages and no of users, meaning no of messages will increase with incresing no of thread users involved.

In [ ]:
Fig2b = info_post_df.groupby('num_users').size().reset_index()
Fig2b.rename(columns={0 :'no_of_messages'}, inplace=True )
Fig2b.head()

In [ ]:
Figure2a = sns.scatterplot(x = "no_of_messages", y = "num_users", data=Fig2b)
rcParams['figure.figsize'] = 6.0,4.0
Figure2a.set_yscale('log')
Figure2a.set_xscale('log')
plt.title("Log plot of - size of threads vs. No of users")
plt.ylabel("Number of Users", size=14)
plt.xlabel("Thread Size (No of messages)", size=14)

Figure 3 demonstrate the realtionship between the total no of messages (log scale) per user_type. This plot shows that the highest no of messages posts or comments are done by students follwed by Annonymous users.



In [ ]:
Fig3_df = df_post.groupby('user_type')['post_id'].count().to_frame().reset_index()
Fig3_df = Fig3_df.sort_values('post_id',ascending = False)
sns.barplot(x= 'user_type',y = 'post_id', data = Fig3_df, color = 'b')
plt.title("Total no of messages (log scale) by Courseera User type ")
plt.ylabel("Number of messages", size=14)
plt.xlabel("User Type", size=14)
plt.xticks(rotation=90, horizontalalignment="center")
plt.yscale('log')




**Figure 4: Scatter plot of Assignments vs.
Study Groups threads represented only with
their number of posts and comments**

As per the research paper figure 4 shows the relationships between post and comments (for Assignments and Study Groups).

Here, no of messages are comprised of posts and comments. And in the dataset only "post_id" atribute is avaialble in order to get count of messages, by no means i could distinguish between post and commnets and get count of in order to establish relationship between posts and commnets.

**Figure 6: Total Number of signed student
messages vs. number of anonymous ones.**

Research paper mentions "The course duration in the dataset vary from 5 to 17
weeks. In order to
aggregate data from courses of different durations, we normalize
and quantize the duration of each course to 20 uniform
subintervals in [0, 1], where: 0 indicates the beginning
of a course and 1 indicates the end of a course".

In order to replicate the results, i normalized the course duration (weeks columns ) and quantize the duration and quantize the duration of each course to 20 uniform subintervals.

quantizing the course duration in 20 intervals gave very fluctuating graph of (course duration vs no of messages (for student and Annonymous users)) and was not giving any imporant info, So i decided to to qunatize course durantion in 7-8 intervals in order to get as similar results given in the research paper.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
info_post_scaled_df = info_post_df
features_to_normalize = ['weeks']
info_post_scaled_df[features_to_normalize] = info_post_scaled_df[features_to_normalize].apply(lambda x:(x-x.min()) / (x.max()-x.min()))

In [ ]:
user_type = ['Student', 'Anonymous']
info_post_scaled_df.user_type.isin(user_type)
info_post_scaled_df =info_post_scaled_df[info_post_scaled_df.user_type.isin(user_type)]

In [ ]:
ranges = np.arange(0.0, 1.1, 0.15)
info_post_scaled_df["range"] = pd.cut(info_post_scaled_df['weeks'], ranges)
Fi6_alt = info_post_scaled_df.groupby(["user_type", "range"]).size().reset_index()
#Fi6_alt = info_post_scaled_df.groupby(["user_type", "range"]).size().reset_index()
Fi6_alt.rename(columns={0 :'msg_alt'}, inplace=True )
Fi6_alt["interval"] = Fi6_alt["range"].apply(lambda x: x.right)
#Fi6_alt


As per the research paper, 

In [ ]:
sns.lineplot(data=Fi6_alt, x="interval", y="msg_alt", hue ='user_type',markers=True, dashes=False)
rcParams['figure.figsize'] = 4.0,3.0
plt.title("Total Number of signed student messages vs. number of anonymous ones.", size =16)
plt.ylabel("Number of Messages", size=14)
plt.xlabel("Normalized course duration", size=14)


**Figure 6 Conclusion: **

As per the research paper, no of messages decreses exponentially till the 6/10 the week of the course. then it reamins steady .

From the above graph it can be seen that that the trend of no of messages is decresing sharply ftill 7/10 duration of the course and then it remains steady and again can see no of messages gradually incresed at the end of the course.

**Figure 5 A: Total number of messages per subforum
over normalized course duration (for quantitative courses)**


Research paper mentions "The course duration in the dataset vary from 5 to 17 weeks. In order to aggregate data from courses of different durations, we normalize and quantize the duration of each course to 20 uniform subintervals in [0, 1], where: 0 indicates the beginning of a course and 1 indicates the end of a course".

In order to replicate the results, i normalized the course duration (weeks columns ) and quantize the duration and quantize the duration of each course to 20 uniform subintervals.

quantizing the course duration in 20 intervals gave very fluctuating graph of (course duration vs no of messages (for quatatiev and non quantative subjects)) and was not giving any imporant info, So i decided to to qunatize course durantion in 7-8 intervals in order to get as similar results given in the research paper.



In [ ]:
thread_post_df = pd.merge(df_thread, df_post, on ="thread_id", how = "left")

In [ ]:
df_info_quan = df_info[df_info.type == 'Quntatve']
df_info_nonquan = df_info[df_info.type == 'Non Quantative']

In [ ]:
thread_post_quan_df = pd.merge(df_info_quan[['course_id', 'weeks']], thread_post_df[['post_id','forum_id_x','course_id_x']],  left_on ='course_id', right_on ='course_id_x', how = "left")
thread_post_nonquan_df = pd.merge(df_info_nonquan[['course_id', 'weeks']], thread_post_df[['post_id','forum_id_x','course_id_x']],  left_on ='course_id', right_on ='course_id_x', how = "left")

In [ ]:
features_to_normalize1 = ['weeks']
thread_post_quan_df[features_to_normalize1] = thread_post_quan_df[features_to_normalize1].apply(lambda x:(x-x.min()) / (x.max()-x.min()))
thread_post_nonquan_df[features_to_normalize1] = thread_post_nonquan_df[features_to_normalize1].apply(lambda x:(x-x.min()) / (x.max()-x.min()))


In [ ]:
ranges1 = np.arange(0.0, 1.1, 0.2)
rcParams['figure.figsize'] = 12.0,6.0
thread_post_quan_df["ranges1"] = pd.cut(thread_post_quan_df['weeks'], ranges1)
Fig5a = thread_post_quan_df.groupby(["forum_id_x", "ranges1"]).size().reset_index()
Fig5a.rename(columns={0 :'No_of_message'}, inplace=True )
Fig5a["interval"] = Fig5a["ranges1"].apply(lambda x: x.right)


In [ ]:
sns.lineplot(data=Fig5a, x="interval", y="No_of_message", hue ='forum_id_x',  markers=True, dashes=False)
plt.title("Total number of messages per subforumover normalized course duration for Quantitative courses", size =16)
plt.ylabel("Number of Messages", size=14)
plt.xlabel("Normalized course duration", size=14)

**Figure 5 A: Total number of messages per subforum
over normalized course duration (quantative Courses)**


> **Conclusion: ** As per the result paper, the Assignments subforums get the relatively
largest amount of posts after the beginning of the course and
they keep a steadier number of posts throughout the course
w.r.t. other subforums. Similar can be inferred from the above graph.

In [ ]:
ranges1 = np.arange(0.0, 1.1, 0.2)
#rcParams['figure.figsize'] = 12.0,6.0
thread_post_nonquan_df["ranges1"] = pd.cut(thread_post_nonquan_df['weeks'], ranges1)
Fig5b = thread_post_quan_df.groupby(["forum_id_x", "ranges1"]).size().reset_index()
Fig5b.rename(columns={0 :'No_of_message'}, inplace=True )
Fig5b["interval"] = Fig5b["ranges1"].apply(lambda x: x.right)

In [1]:
Fig5b

NameError: ignored

In [ ]:
sns.lineplot(data=Fig5b, x="interval", y="No_of_message", hue ='forum_id_x',  markers=True, dashes=False)


**Figure 5 A: Total number of messages per subforum
over normalized course duration (Non quantative Courses)**


**Conclusion** As per the research paper, we  notice (as from
other analyses) a slight peak of activity around the 85% of
the course duration. Similar results can b einferred from the above graph.
